In [1]:
___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor\n°OoO_FXCM_Oo0°\nLumberJack Jyss 5779(c)')
print('GOLEM Version v1.33')
print('Importing librairies...')
import forexconnect
from forexconnect import fxcorepy
from forexconnect import ForexConnect, ResponseListener, Common
import time
import numpy as np
import pandas as pd
from colorama import Fore, Back, Style 
import datetime as dt
import talib
from zigzag import *
#import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
#from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,roc_curve,confusion_matrix,classification_report
print('Librairies imported')
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


def session_status_changed(session: fxcorepy.O2GSession,
                           status: fxcorepy.AO2GSessionStatus.O2GSessionStatus):
    print("Trading session status: " + str(status))
    return

pos_long = 0
pos_short = 0

# TOKEN = 'e053ac1597cef331df9429ac8151100ea9f1c411'
# server = 'demo'
user_id = 'D261155614'
password = '3160'
url = 'fxcorporate.com/Hosts.jsp'
accountid='01147539'

# minutes: 'm1' , 'm5' , 'm15' , 'm30' 
# hours: 'H1' , 'H2', 'H3', 'H4', 'H6' 'H8'
# one day: 'D1'
# one week: 'W1'
# one month: 'M1'

ticker = 'GER30'
period = 'm1'
number = 1000
long_open = 95
long_close = 95
short_open = 95
short_close = 95
jet_lag = -2
pik_long = 0.001
pik_short = 0.001
targetup = 0.1
targetdown = 0.1
amount = 30
buysell = None
side = None
trade_id = None
open_rate = None
open_time = None
offerid = None
noo_down = 0
noo_up = 0
noo_target = 0
# Time Windows
# start = dt.datetime(2017, 7, 15)
# stop = dt.datetime(2017, 8, 1)
# con.get_candles('EUR/USD', period='D1',start=start, stop=stop)

buy_price = 0
sell_price = 0
pnl = 0
#try :

fx = ForexConnect()
fx.login(user_id, password, url,"Demo", session_status_callback=session_status_changed)
table_manager = fx.table_manager
trades_table = table_manager.get_table(fx.TRADES)


for row in trades_table:
    trade =None
    side = row['buy_sell']
    pnl = row['pl']
    trade_id = row['trade_id']
    open_rate = row['open_rate']
    should_close = row['close']
    amount = row['amount']

print('\r Side',side,'pnl : ',pnl,'Trade_ID : ',trade_id,'Open_Rate',open_rate,'Amount',amount)
if side == 'B':
    pos_long = 1
elif side == 'S':
    pos_short = 1

# SCRAP DATA
tmps1=time.time()
history = fx.get_history(ticker, period,quotes_count=number)
data = pd.DataFrame({'Date':history['Date'],'Open':history['BidOpen'],'High':history['BidHigh'],'Low':history['BidLow'],'Close':history['BidClose']})
df = data.copy()
tmps2=round(time.time()-tmps1,2)

print('\033[33m' + 'Initializing GOLEM...\n')
print('\033[35m' +'Long Pos =',pos_long)
print('Short Pos =',pos_short,Style.RESET_ALL)

# DATA PREPARATION

tmps1=time.time()
print('df.shape control : ',df.shape)
df.reset_index(inplace=True,drop=True)
dataset_1D = df.copy()
dataset_1D = dataset_1D.dropna()
rsi = talib.RSI(dataset_1D['Close'],timeperiod=2)
upper, middle, lower =  talib.BBANDS(dataset_1D['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
sma5 = talib.SMA(dataset_1D['Close'],timeperiod=5)
sma8 = talib.SMA(dataset_1D['Close'],timeperiod=8)
sma10 = talib.SMA(dataset_1D['Close'],timeperiod=10)
sma12 = talib.SMA(dataset_1D['Close'],timeperiod=12)
sma15 = talib.SMA(dataset_1D['Close'],timeperiod=15)
sma30 = talib.SMA(dataset_1D['Close'],timeperiod=30)
sma35 = talib.SMA(dataset_1D['Close'],timeperiod=35)
sma40 = talib.SMA(dataset_1D['Close'],timeperiod=40)
sma45 = talib.SMA(dataset_1D['Close'],timeperiod=45)
sma50 = talib.SMA(dataset_1D['Close'],timeperiod=50)
aroondown, aroonup = talib.AROON(dataset_1D['High'], dataset_1D['Low'], timeperiod=25)
aroon = aroonup - aroondown #(aroonup-aroondown)/abs((aroonup-aroondown))

delta5_8 = sma5 - sma8
delta8_10 = sma8 - sma10
delta10_12 = sma10 - sma12
delta12_15 = sma12 - sma15
delta15_30 = sma15 - sma30
delta30_35 = sma30 - sma35
delta35_40 = sma35 - sma40
delta40_45 = sma40 - sma45
delta45_50 = sma45 - sma50

X = dataset_1D['Close']
pivots = peak_valley_pivots(X.values, pik_long, -pik_short)
ts_pivots = pd.Series(X, index=X.index)
ts_pivots = ts_pivots[pivots != 0]

peak = []
valley = []

for i in range(0,dataset_1D.shape[0]):
    if pivots[i] == 1:
        peak.append(pivots[i])
        valley.append(0)
    elif pivots[i] == -1:
        peak.append(0)
        valley.append(pivots[i])
    else:
        peak.append(0)
        valley.append(0)

bbdelta = upper - middle
price_bolup = dataset_1D['Close'] - lower
price_bolow = dataset_1D['Close'] - upper

rsi5_list = []
rsi95_list = []
for i in range(0,dataset_1D.shape[0]):
    try:
        rsi95_list.append(95 - rsi[i])
        rsi5_list.append(rsi[i] - 5)
    except:
        rsi95_list.append(0)
        rsi5_list.append(0)
varop_spy = dataset_1D['Open'] - dataset_1D['Close']
varhl_spy = dataset_1D['High'] - dataset_1D['Low']
dataset_1D['Varop_Spy'] = varop_spy
dataset_1D['Varhl_spy'] = varhl_spy
dataset_1D['RSI'] = rsi
dataset_1D['Aroon'] = aroon
dataset_1D['95 - RSI'] = np.array(rsi95_list)
dataset_1D['RSI - 5'] = np.array(rsi5_list)
dataset_1D['BBD_Delta_Up'] = bbdelta
dataset_1D['delta5_8'] = delta5_8
dataset_1D['delta8_10'] = delta8_10
dataset_1D['delta10_12'] = delta10_12
dataset_1D['delta12_15'] = delta12_15
dataset_1D['delta15_30'] = delta15_30
dataset_1D['delta30_35'] = delta30_35
dataset_1D['delta35_40'] = delta35_40
dataset_1D['delta40_45'] = delta40_45
dataset_1D['delta45_50'] = delta45_50
dataset_1D['Peaks'] = abs(np.array(peak))
dataset_1D['Valley'] = abs(np.array(valley))
tsf = talib.TSF(dataset_1D['Close'],timeperiod=14)
delta_tsf = dataset_1D['Close'] - tsf
dataset_1D['delta_tsf'] = tsf
forosc = []
forosc.append(0)
for i in range(1,dataset_1D.shape[0]):
    try:
        forosc.append((dataset_1D.iloc[i,4] - tsf[i-1])*100/dataset_1D.iloc[i,4])
    except:
        forosc.append(0)
dataset_1D['Forcast Oscillator'] = (forosc)
target_up = []
target_down = []

for i in range(0,dataset_1D.shape[0]-5):

    if (dataset_1D.iloc[i+1,5] * dataset_1D.iloc[i,5]) < 0 :
        if (dataset_1D.iloc[i+1,4] - dataset_1D.iloc[i,4]) > targetup : 
            target_up.append(1)
            target_down.append(0)
        elif (dataset_1D.iloc[i+1,4] - dataset_1D.iloc[i,4]) < -targetdown: 
            target_up.append(0)
            target_down.append(1)
        else:
            target_up.append(0)
            target_down.append(0)
    else :
        target_up.append(0)
        target_down.append(0) 

target_up.append(0)
target_down.append(0)
target_up.append(0)
target_down.append(0)
target_up.append(0)
target_down.append(0)
target_up.append(0)
target_down.append(0)
target_up.append(0)
target_down.append(0)
dataset_1D['target_up'] = target_up  # target_up # abs(np.array(valley))#target_up
dataset_1D['target_down'] = target_down # target_down # abs(np.array(peak))#target_down
dataset_1D['target_value'] = dataset_1D['Close']
dataset_1D = dataset_1D.drop(['Open','High','Low','Close'],axis=1)
df_1D = dataset_1D.copy()
df_1D = df_1D.iloc[:,1:]
df_1D['Date'] = df['Date']
df_1D = df_1D.dropna()
df_1D = df_1D[['Date'] + df_1D.columns[:-1].tolist()]
df_1D.reset_index(inplace=True,drop=True)

print('\033[36m'+'GOLEM begins Computing...'+Style.RESET_ALL)



print('Computing done')

# TEST POS

print('\n Aroon : ',aroon.iloc[-1],end='')
print('\n Aroon Up : ',aroonup.iloc[-1],end='')
print('\n Aroon Down : ',aroondown.iloc[-1],end='')
print('\n Long Pos =',pos_long,' ',end='')
print('\n Short Pos =',pos_short,' ',end='')
print('\n Initializing GOLEM on STREAMING LIVE...',' ',end='')
print('\n Long Pos =',pos_long,' ',end='')
print('\n Short Pos =',pos_short,' ',end='')

# UPDATE DATA
print('\nInstrument :', ticker,'\n')
print (' - Side',side,'\n pnl : ',pnl,'\n Trade_ID : ',trade_id,'\n Open_Rate',open_rate,'\n Amount',amount,
           ' -  Bid %f' % fx.get_history(ticker, 't1',quotes_count=1)['Bid']," -  Ask %f" % fx.get_history(ticker, 't1',quotes_count=1)['Ask'],
           '\033[43m'+' Tempo '+'\033[0m'+'\033[46m'+'...'+'\033[0m')
if pos_long == 1:
    if pnl > 0 :
        print('\r \033[42m pnl : ',round(pnl,2),'\033[0m',end='')
    elif pnl < 0:
        print('\r \033[41m pnl : ',round(pnl,2),'\033[0m',end='')
elif pos_short == 1:
    if pnl > 0 :
        print('\r \033[42m pnl : ',round(pnl,2),'\033[0m',end='')
    elif pnl < 0:
        print('\r\033[41m pnl : ',round(pnl,2),'\033[0m',end='')
else:
    print(' \r - No position',end='',flush=True)
for loop in range(0,1000):
    #print(' \n INFRA LOOP ',loop,'\n')
    tmps1=time.time()
    
    history = fx.get_history(ticker, period,quotes_count=1)
    last = pd.DataFrame({'Date':history['Date'],'Open':history['BidOpen'],'High':history['BidHigh'],'Low':history['BidLow'],'Close':history['BidClose']})
    

    if last.iloc[-1,0].minute == df.iloc[-1,0].minute:
        
        print ('\r',dt.datetime.now().hour,'H',dt.datetime.now().minute,'min',dt.datetime.now().second,'sec',end='',flush=True)
        
        for row in trades_table:
            trade =None
            side = row['buy_sell']
            pnl = row['pl']
            trade_id = row['trade_id']
            open_rate = row['open_rate']
            should_close = row['close']
            amount = row['amount']
        
    else :
        df = df.iloc[1:,:]
        df = df.append({'Date':last.iloc[-1,0],'Open':last.iloc[-1,1], 'High':last.iloc[-1,2],'Low':last.iloc[-1,3], 'Close':last.iloc[-1,4]}, ignore_index=True)

        tmps1=time.time()
        print('df.shape control : ',df.shape)
        df.reset_index(inplace=True,drop=True)
        dataset_1D = df.copy()
        dataset_1D = dataset_1D.dropna()
        rsi = talib.RSI(dataset_1D['Close'],timeperiod=2)
        upper, middle, lower =  talib.BBANDS(dataset_1D['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
        sma5 = talib.SMA(dataset_1D['Close'],timeperiod=5)
        sma8 = talib.SMA(dataset_1D['Close'],timeperiod=8)
        sma10 = talib.SMA(dataset_1D['Close'],timeperiod=10)
        sma12 = talib.SMA(dataset_1D['Close'],timeperiod=12)
        sma15 = talib.SMA(dataset_1D['Close'],timeperiod=15)
        sma30 = talib.SMA(dataset_1D['Close'],timeperiod=30)
        sma35 = talib.SMA(dataset_1D['Close'],timeperiod=35)
        sma40 = talib.SMA(dataset_1D['Close'],timeperiod=40)
        sma45 = talib.SMA(dataset_1D['Close'],timeperiod=45)
        sma50 = talib.SMA(dataset_1D['Close'],timeperiod=50)
        aroondown, aroonup = talib.AROON(dataset_1D['High'], dataset_1D['Low'], timeperiod=25)
        aroon = aroonup - aroondown #(aroonup-aroondown)/abs((aroonup-aroondown))

        delta5_8 = sma5 - sma8
        delta8_10 = sma8 - sma10
        delta10_12 = sma10 - sma12
        delta12_15 = sma12 - sma15
        delta15_30 = sma15 - sma30
        delta30_35 = sma30 - sma35
        delta35_40 = sma35 - sma40
        delta40_45 = sma40 - sma45
        delta45_50 = sma45 - sma50

        X = dataset_1D['Close']
        pivots = peak_valley_pivots(X.values, pik_long, -pik_short)
        ts_pivots = pd.Series(X, index=X.index)
        ts_pivots = ts_pivots[pivots != 0]

        peak = []
        valley = []

        for i in range(0,dataset_1D.shape[0]):
            if pivots[i] == 1:
                peak.append(pivots[i])
                valley.append(0)
            elif pivots[i] == -1:
                peak.append(0)
                valley.append(pivots[i])
            else:
                peak.append(0)
                valley.append(0)

        bbdelta = upper - middle
        price_bolup = dataset_1D['Close'] - lower
        price_bolow = dataset_1D['Close'] - upper

        rsi5_list = []
        rsi95_list = []
        for i in range(0,dataset_1D.shape[0]):
            try:
                rsi95_list.append(95 - rsi[i])
                rsi5_list.append(rsi[i] - 5)
            except:
                rsi95_list.append(0)
                rsi5_list.append(0)
        varop_spy = dataset_1D['Open'] - dataset_1D['Close']
        varhl_spy = dataset_1D['High'] - dataset_1D['Low']
        dataset_1D['Varop_Spy'] = varop_spy
        dataset_1D['Varhl_spy'] = varhl_spy
        dataset_1D['RSI'] = rsi
        dataset_1D['Aroon'] = aroon
        dataset_1D['Aroon UP'] = aroonup
        dataset_1D['Aroon Down'] = aroondown
        dataset_1D['95 - RSI'] = np.array(rsi95_list)
        dataset_1D['RSI - 5'] = np.array(rsi5_list)
        dataset_1D['BBD_Delta_Up'] = bbdelta
        dataset_1D['delta5_8'] = delta5_8
        dataset_1D['delta8_10'] = delta8_10
        dataset_1D['delta10_12'] = delta10_12
        dataset_1D['delta12_15'] = delta12_15
        dataset_1D['delta15_30'] = delta15_30
        dataset_1D['delta30_35'] = delta30_35
        dataset_1D['delta35_40'] = delta35_40
        dataset_1D['delta40_45'] = delta40_45
        dataset_1D['delta45_50'] = delta45_50
        dataset_1D['Peaks'] = abs(np.array(peak))
        dataset_1D['Valley'] = abs(np.array(valley))
        tsf = talib.TSF(dataset_1D['Close'],timeperiod=14)
        delta_tsf = dataset_1D['Close'] - tsf
        dataset_1D['delta_tsf'] = tsf
        forosc = []
        forosc.append(0)
        for i in range(1,dataset_1D.shape[0]):
            try:
                forosc.append((dataset_1D.iloc[i,4] - tsf[i-1])*100/dataset_1D.iloc[i,4])
            except:
                forosc.append(0)
        dataset_1D['Forcast Oscillator'] = (forosc)
        target_up = []
        target_down = []

        for i in range(0,dataset_1D.shape[0]-5):

            if (dataset_1D.iloc[i+1,5] * dataset_1D.iloc[i,5]) < 0 :
                if (dataset_1D.iloc[i+1,4] - dataset_1D.iloc[i,4]) > targetup : 
                    target_up.append(1)
                    target_down.append(0)
                elif (dataset_1D.iloc[i+1,4] - dataset_1D.iloc[i,4]) < -targetdown: 
                    target_up.append(0)
                    target_down.append(1)
                else:
                    target_up.append(0)
                    target_down.append(0)
            else :
                target_up.append(0)
                target_down.append(0) 

        target_up.append(0)
        target_down.append(0)
        target_up.append(0)
        target_down.append(0)
        target_up.append(0)
        target_down.append(0)
        target_up.append(0)
        target_down.append(0)
        target_up.append(0)
        target_down.append(0)
        dataset_1D['target_up'] = target_up  # target_up # abs(np.array(valley))#target_up
        dataset_1D['target_down'] = target_down # target_down # abs(np.array(peak))#target_down
        dataset_1D['target_value'] = dataset_1D['Close']
        dataset_1D = dataset_1D.drop(['Open','High','Low','Close'],axis=1)
        df_1D = dataset_1D.copy()
        df_1D = df_1D.iloc[:,1:]
        df_1D['Date'] = df['Date']
        df_1D = df_1D.dropna()
        df_1D = df_1D[['Date'] + df_1D.columns[:-1].tolist()]
        df_1D.reset_index(inplace=True,drop=True)



        # TESTING ONLY
        print()
        print('GOLEM begins testing...')

        tmps2=round(time.time()-tmps1,2)
        print ("Testing executed in = %f" %tmps2,'secondes')
        print('Time of request', Fore.BLUE)
        print(dt.datetime.now().hour+jet_lag,'H',dt.datetime.now().minute,'min',dt.datetime.now().second,'seconds',Style.RESET_ALL)
        print('\033[45m')
        print(Style.RESET_ALL)

        try:    
            for row in trades_table:
                trade =None
                side = row['buy_sell']
                pnl = row['pl']
                trade_id = row['trade_id']
                open_rate = row['open_rate']
                should_close = row['close']
                amount = row['amount']

            if pos_long == 1:
                if pnl > 0 :
                    print('\033[42m pnl : ',round(pnl,2),'\033[0m')
                elif pnl < 0:
                    print('\033[41m pnl : ',round(pnl,2),'\033[0m')
            elif pos_short == 1:
                if pnl > 0 :
                    print('\033[42m pnl : ',round(pnl,2),'\033[0m')
                elif pnl < 0:
                    print('\033[41m pnl : ',round(pnl,2),'\033[0m')

            print('Aroon : ',aroon.iloc[-1])
            print('Aroon Up : ',aroonup.iloc[-1])
            print('Aroon Down : ',aroondown.iloc[-1])
            print('Long Pos =',pos_long)
            print('Short Pos =',pos_short)

        except :
            pass
        # ACTIONS

        if aroonup.iloc[-1] < 5 and aroondown.iloc[-1] > 95:

            if  pos_long == 0 and pos_short == 1 :
                buysell = 'B'
                try:
                    request_close = fx.create_order_request(
                   order_type='C',
                   ACCOUNT_ID=accountid,
                    OFFER_ID = offerid,
                   BUY_SELL=buysell,
                   AMOUNT=amount,
                   RATE=should_close+0.5,
            TIME_IN_FORCE='IOC',
            TRADE_ID = trade_id
               )
                    resp_close = fx.send_request(request_close)

                    closed_table = table_manager.get_table(fx.CLOSED_TRADES)
                    for row in closed_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        amount = row['amount']
                        open_time = row['open_time']
                        close_rate = row['close_rate']
                        close_time = row['close_time']

                    print('\033[47m \033[31m Close Short avec un pnl de', pnl,'\033[0m')
                    print('Résumé : ')
                    print(' Open date : ', open_time)
                    print('Open Rate : ',open_rate)
                    print('Amount : ',amount)
                    print('Close date : ',close_time)
                    print('Close Rate : ',close_rate)
                    print('')
                    pos_short = 0
                    noo_down = 0
                    noo_up = 0
                    continue
                except:
                    print("WARNING! It wasn't possible to close the position" )
                    pass

            if pos_long == 0 and pos_short == 0:

                buysell = 'B'
                trade_rate = last.iloc[-1,-1] + 3
                print('Trying Long at :',trade_rate)
                try:
                    request_open = fx.create_order_request(SYMBOL=ticker,
                   order_type='O',
                   ACCOUNT_ID=accountid,
                   BUY_SELL=buysell,
                   AMOUNT=amount,
                   RATE=trade_rate,
            TIME_IN_FORCE='IOC'
               )
                    resp_open = fx.send_request(request_open)
                    order_id = resp_open.order_id
                    offer = Common.get_offer(fx,ticker)
                    offerid = offer.offer_id
                    table_manager = fx.table_manager
                    trades_table = table_manager.get_table(fx.TRADES)
                    for row in trades_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        should_close = row['close']
                        amount = row['amount']
                    print('Side',side,'pnl : ',pnl,'Trade_ID : ',trade_id,'Open_Rate',open_rate,'Amount',amount)

                    print('Résumé : ')
                    print(' Open date : ', open_time)
                    print('Open Rate : ',open_rate)
                    print('Amount : ',amount)
                    print('pnl : ',pnl)
                    print('                                                     Go Long!')
                    print('')
                    pos_long = 1
                    noo_down = 0
                    noo_up = 0
                except:
                    print("WARNING! It wasn't possible to open the position - Price instability detected - " )
                    pass

        elif aroondown.iloc[-1] < 5 and aroonup.iloc[-1] > 95:
            noo_down = noo_down + 1
            noo_up = 0
            if  pos_long == 1 and pos_short == 0 and  noo_down > noo_target:
                buysell = 'S'
                try:
                    request_close = fx.create_order_request(
                   order_type='C',
                   ACCOUNT_ID=accountid,
                    OFFER_ID = offerid,
                   BUY_SELL=buysell,
                   AMOUNT=amount,
                   RATE=should_close-0.5,
            TIME_IN_FORCE='IOC',
            TRADE_ID = trade_id
               )
                    resp_close = fx.send_request(request_close)

                    closed_table = table_manager.get_table(fx.CLOSED_TRADES)
                    for row in closed_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        amount = row['amount']
                        open_time = row['open_time']
                        close_rate = row['close_rate']
                        close_time = row['close_time']

                    print('\033[47m \033[31m Close Long avec un pnl de', pnl,'\033[0m')
                    print('Résumé : ')
                    print(' Open date : ', open_time)
                    print('Open Rate : ',open_rate)
                    print('Amount : ',amount)
                    print('Close date : ',close_time)
                    print('Close Rate : ',close_rate)
                    pos_long = 0
                    noo_down = 0
                    noo_up = 0
                    continue
                except:
                    print("WARNING! It wasn't possible to close the position" )
                    continue

            if pos_long == 0 and pos_short == 0:
                buysell = 'S'
                trade_rate = last.iloc[-1,-1] - 3
                print('Trying short at : ',trade_rate)
                try:
                    request_open = fx.create_order_request(SYMBOL=ticker,
                   order_type='O',
                   ACCOUNT_ID=accountid,
                   BUY_SELL=buysell,
                   AMOUNT=amount,
                   RATE=trade_rate,
            TIME_IN_FORCE='IOC'
               )
                    resp_open = fx.send_request(request_open)
                    order_id = resp_open.order_id
                    offer = Common.get_offer(fx,ticker)
                    offerid = offer.offer_id
                    table_manager = fx.table_manager
                    trades_table = table_manager.get_table(fx.TRADES)
                    for row in trades_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        should_close = row['close']
                        amount = row['amount']
                    print('Side',side,'pnl : ',pnl,'Trade_ID : ',trade_id,'Open_Rate',open_rate,'Amount',amount)

                    print('Résumé : ')
                    print(' Open date : ', open_time)
                    print('Open Rate : ',open_rate)
                    print('Amount : ',amount)
                    print('pnl : ',pnl)
                    print('')
                    print(                                                    'Go Short!')
                    pos_short = 1
                    noo_down = 0
                    noo_up = 0
                except:
                    print("WARNING! It wasn't possible to open the position - Price instability detected" )
                    continue

Global Optimized LumberJack Environment Motor
°OoO_FXCM_Oo0°
LumberJack Jyss 5779(c)
GOLEM Version v1.33
Importing librairies...


Using TensorFlow backend.


Librairies imported
Trading session status: CONNECTING
Trading session status: CONNECTED
 Side None pnl :  0 Trade_ID :  None Open_Rate None Amount 30
Initializing GOLEM...

Long Pos = 0
Short Pos = 0 
df.shape control :  (1000, 5)
GOLEM begins Computing...
Computing done

 Aroon :  36.0
 Aroon Up :  92.0
 Aroon Down :  56.0
 Long Pos = 0  
 Short Pos = 0  
 Initializing GOLEM on STREAMING LIVE...  
 Long Pos = 0  
 Short Pos = 0  
Instrument : GER30 

 - Side None 
 pnl :  0 
 Trade_ID :  None 
 Open_Rate None 
 Amount 30  -  Bid 12630.050000  -  Ask 12630.990000  Tempo ...
 11 H 21 min 17 secdf.shape control :  (1000, 5)

GOLEM begins testing...
Testing executed in = 0.170000 secondes
Time of request 
9 H 21 min 17 seconds 


Aroon :  36.0
Aroon Up :  88.0
Aroon Down :  52.0
Long Pos = 0
Short Pos = 0
 11 H 22 min 12 secdf.shape control :  (1000, 5)

GOLEM begins testing...
Testing executed in = 0.140000 secondes
Time of request 
9 H 22 min 12 seconds 


Aroon :  36.0
Aroon Up :  84.